In [1]:
import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import re
import seaborn as sns
import matplotlib.pyplot  as plt
%matplotlib inline
import requests

### Data Files

Link to Download CSVs: https://drive.google.com/drive/folders/1o5gQKHexz1-igyKOkyeiCnVCPQgjq0C-?usp=sharing

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

In [3]:
#Directory
path = os.getcwd() 
data_root = path + '\\DATA\\'
data_root

'C:\\Users\\Kaipable\\Desktop\\GitHub\\voter-protection-corps\\pollworkers_src\\DATA\\'

##  Common Congressional Election Survey Data

In [4]:
# Read in CCES data
cces = pd.read_csv(data_root + 'CCES16_Common_OUTPUT_Feb2018_VV.tab', sep='\t')

#Explore 
print(cces.shape)
print(cces.columns)
cces.head(5)

C:\Users\Kaipable\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (46,47,48,49,50,51,52,53,54,55,370,371,373,374,375,376,377,378,379,380,381,382,383,385,386,388,389,390,391,393,394,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,419,420,422,423,424,425,426,427,430,432,436,439,441,443,445,446,447,448,449,450,451,452,453,454,455,456,458,460,462,486,487,493,494,513,515,520,522,526,527,528,533,534,535,541,542,543,544,545,559,560) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(64600, 563)
Index(['V101', 'commonweight_vv', 'commonweight_vv_post', 'commonweight_vv_lgbt', 'commonweight', 'commonweight_post', 'tookpost', 'lookupzip', 'inputstate', 'cdid113',
       ...
       'CL_E2016PPEP', 'CL_E2016PPVM', 'CL_E2016PEP', 'CL_E2016PVM', 'starttime_pre', 'endtime_pre', 'starttime_post', 'endtime_post', 'dma', 'dmaname'], dtype='object', length=563)


,V101,commonweight_vv,commonweight_vv_post,commonweight_vv_lgbt,commonweight,commonweight_post,tookpost,lookupzip,inputstate,cdid113,cdid115,countyfips,countyname,CurrentGovName,CurrentGovParty,CurrentHouseGender,CurrentHouseName,CurrentHouseParty,CurrentSen1Gender,CurrentSen1Name,CurrentSen1Party,CurrentSen2Gender,CurrentSen2Name,CurrentSen2Party,GovCand1Incumbent,GovCand1Name,GovCand1Party,GovCand2Incumbent,GovCand2Name,GovCand2Party,GovCand3Name,GovCand3Party,HouseCand1Incumbent,HouseCand1Name,HouseCand1Party,HouseCand2Incumbent,HouseCand2Name,HouseCand2Party,HouseCand3Name,HouseCand3Party,HouseCand4Name,HouseCand4Party,HouseCand5Name,HouseCand5Party,HouseCand6Name,HouseCand6Party,HouseCand7Name,HouseCand7Party,HouseCand8Name,HouseCand8Party,HouseCand9Name,HouseCand9Party,HouseCand10Name,HouseCand10Party,HouseCand11Name,HouseCand11Party,LegName,LowerChamberName,SenCand1Incumbent,SenCand1Name,SenCand1Party,SenCand2Incumbent,SenCand2Name,SenCand2Party,SenCand3Name,SenCand3Party,SenCand4Name,SenCand4Party,CCEStake,comptype,add_confirm,votereg,votereg_f,regzip,inputzip,birthyr,gender,educ,race,race_other,hispanic,marstat,multrace_1,multrace_2,multrace_3,multrace_4,multrace_5,multrace_8,multrace_97,multrace_98,multrace_99,Hispanic_origin_1,Hispanic_origin_2,Hispanic_origin_3,Hispanic_origin_4,Hispanic_origin_5,Hispanic_origin_6,Hispanic_origin_7,Hispanic_origin_8,Hispanic_origin_9,Hispanic_origin_10,Hispanic_origin_11,Hispanic_origin_12,Hispanic_origin_t,Asian_origin_1,Asian_origin_2,Asian_origin_3,Asian_origin_4,Asian_origin_5,Asian_origin_6,Asian_origin_7,Asian_origin_8,Asian_origin_9,Asian_origin_10,Asian_origin_11,Asian_origin_12,Asian_origin_13,Asian_origin_14,Asian_origin_15,Asian_origin_t,ccesmodule,CC16_300_1,CC16_300_2,CC16_300_3,CC16_300_4,CC16_300_5,CC16_300_6,CC16_300b,CC16_300c,CC16_300d_1,CC16_300d_2,CC16_300d_3,CC16_300d_4,CC16_300d_5,CC16_301a,CC16_301b,CC16_301c,CC16_301d,CC16_301e,CC16_301f,CC16_301g,CC16_301h,CC16_301i,CC16_301j,CC16_301k,CC16_301l,CC16_301m,CC16_301n,CC16_301o,CC16_302,CC16_303,CC16_304,CC16_305_1,CC16_305_2,CC16_305_3,CC16_305_4,CC16_305_5,CC16_305_6,CC16_305_7,CC16_305_8,CC16_305_9,CC16_305_10,CC16_305_11,CC16_307,CC16_312_1,CC16_312_2,CC16_312_3,CC16_312_4,CC16_312_5,CC16_312_6,CC16_312_7,CC16_321a,CC16_321b,CC16_321c,CC16_321d,CC16_322a,CC16_322b,CC16_322c,CC16_322d,CC16_320a,CC16_320b,CC16_320c,CC16_320d,CC16_320e,CC16_320f,CC16_320g,CC16_320h,CC16_316,CC16_326,CC16_326_t,CC16_327,CC16_328,CC16_330a,CC16_330b,CC16_330d,CC16_330e,CC16_331_1,CC16_331_2,CC16_331_3,CC16_331_4,CC16_331_5,CC16_331_6,CC16_331_7,CC16_331_8,CC16_331_9,CC16_332a,CC16_332b,CC16_332c,CC16_332d,CC16_332e,CC16_332f,CC16_333a,CC16_333b,CC16_333c,CC16_333d,CC16_334a,CC16_334b,CC16_334c,CC16_334d,CC16_335,CC16_337_1,CC16_337_2,CC16_337_3,CC16_340a,CC16_340b,CC16_340c,CC16_340d,CC16_340e,CC16_340f,CC16_340g,CC16_340h,CC16_340i,CC16_340j,CC16_340k,CC16_340l,CC16_340m,CC16_340n,CC16_340o,CC16_340p,CC16_350,CC16_351A,CC16_351B,CC16_351C,CC16_351D,CC16_351E,CC16_351F,CC16_351G,CC16_351H,CC16_351I,CC16_351K,...,religpew_congreg,religpew_congreg_t,religpew_holiness,religpew_holiness_t,religpew_reformed,religpew_reformed_t,religpew_advent,religpew_advent_t,religpew_catholic,religpew_catholic_t,religpew_mormon,religpew_mormon_t,religpew_orthodox,religpew_orthodox_t,religpew_jewish,religpew_jewish_t,religpew_muslim,religpew_muslim_t,religpew_buddhist,religpew_buddhist_t,religpew_hindu,religpew_hindu_t,ideo5,milstat_1,milstat_2,milstat_3,milstat_4,milstat_5,citylength_1,citylength_2,union,unionhh,newsint,faminc,investor,sexuality,trans,healthins_1,healthins_2,healthins_3,healthins_4,healthins_5,healthins_6,healthins2,phone,internethome,internetwork,lookupzip_post,cdid113_post,cdid115_post,inputstate_post,regzip_confirm_post,reszip_confirm_post,inputzip_post,votereg_post,votereg_f_post,regzip_post,countyfips_post,countyname_post,AskParty_post,CurrentGovName_post,CurrentGovParty_post,CurrentHouseGender_post,CurrentHouseName_post,Curr

In [5]:
# Common Field
cces.countyfips[0:5]

0    33011.0
1    22071.0
2    29155.0
3     1047.0
4     8069.0
Name: countyfips, dtype: float64

In [6]:
#Clean Up County Code
cces['clean_county'] = cces['countyfips'].apply(lambda x: str(x).strip('.0')[-3:]   if len(str(x)) > 6 else '0'+ str(x).strip('.0')[0:2])
print(cces['clean_county'][0:5])

# Clean Up State Codes
cces['clean_state'] = cces['inputstate'].apply(lambda x: str(x) if len(str(x)) > 1 else '0'+ str(x))
#print(cces['clean_state'][0:5])

# Altogether;  Cretae FIPS Column
cces['FIPSCode'] = cces['clean_state'] + cces['clean_county']

#Match Type of DF
#cces['FIPSCode']= pd.to_numeric(cces['FIPSCode'], errors='coerce').fillna(0).astype(np.int64)
print(cces['FIPSCode'][0:5])

0    011
1    071
2    155
3    010
4    080
Name: clean_county, dtype: object
0    33011
1    22071
2    29155
3    01010
4    08080
Name: FIPSCode, dtype: object


In [7]:
#Investigate Voter Wait Time Column
percent_null = cces['CC16_404'].isna().sum() / len(cces.index)

print('Of the',len(cces.index), 'rows in the CCES dataset,', np.round(percent_null*100,2), '% of wait time entries are null.')
print('CCES has data from',cces['FIPSCode'].nunique(), 'counties')

#Categorical Data Codes
vote_dict = {1:0, 2: 'Less than 10', 3: '10-30 minutes', 4: '31 minutes - 1 hour', 5: 'more than one hour', 6: None}

Of the 64600 rows in the CCES dataset, 46.82 % of wait time entries are null.
CCES has data from 2408 counties


In [8]:
#DF for Just Vote Times to Merge
vote_times = cces[['V101','inputstate','FIPSCode','CC16_404']]
vote_times['CC16_404'] = vote_times['CC16_404'].replace(6,np.NaN) # 6 equates to NaN
vote_times = vote_times.dropna()

# Interpretable variable names. 
vote_times.columns = ['caseid','state','FIPSCode', 'wait_time'] 

#Did it Work?
vote_times.describe()
vote_times['wait_time'].dtype
print('We have vote times for',vote_times['FIPSCode'].nunique(), 'counties')
vote_times.head()

We have vote times for 2182 counties


C:\Users\Kaipable\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,caseid,state,FIPSCode,wait_time
0,222168628,33,33011,2.0
1,273691199,22,22071,1.0
4,290387662,8,08080,1.0
6,292860642,48,48231,2.0
8,295717127,13,13213,2.0


In [9]:
# Average Times by County
vote_by_fips = vote_times.groupby('FIPSCode', 
                as_index=False)['wait_time'].agg({'wait_time':'mean',
                  'wait_count': 'count'})
vote_by_fips.head()

,FIPSCode,wait_time,wait_count
0,01010,2.250720,347
1,01011,1.953846,130
2,02020,1.840909,44
3,02021,1.695652,23
4,02022,1.000000,2


# EAVS Dataset

#### Poll Worker Age
* D8a. TOTAL number of poll workers  
* D8b. Under 18 years old  
* D8c. 18 to 24  
* D8d. 26 to 40  
* D8e. 41 to 60  
* D8f. 61 to 70  
* D8g. 71 years old and over  

#### Section A: Voter Registration  
General info about registration numbers, forms processed, mode, and decision  
* A1a Total  Registered Voters
* Number of Active (A1b) and Inactive Voters (A1c)  
** Percent inactive (A1c/A1a) could communicate folks whose registration will take longer -- longer wait times > more resources
* % Same Day Registration
* Total # of same day registration (A2a)
* all registered voters (A1a)
* all new registrations (A3b)
* total processed registrations (A3a) in time frame

#### Section D: Total Votes Cast an In-Person Voting  
Includes number of in-person voters, # precincts, # of polling places, # of poll workers, # in-person absentee  
* Total number of votes cast (D1a)  
* Total number voting in-person, physical voters (D2)
* In-person on election day (D2a)
* Early vote in-person (D2b)
* of Precincts (D3)
* Physical Polling Places (D4a)
Early vote (D5a) 
* Non-election office polling places (D5c)    

#### POLL WORKERS  
* Total # Poll workers (D8a)  
For election day (D6)  
For early voting (D7)  
* Ease of Recruiting Poll Workers (D9)  

#### PARTICIPATION  
* F1a. TOTAL number of voters participating
* F1b. Voted at a physical polling place on Election Day  

#### New Vars
* In Person Election Day Votes / Total Cotes Cast = D2 / D1 
    - In Person Election Day VOTERS / Total Voters = F1b/F1a
* Total Number of Poll Workers = D8a


In [10]:
eavs = pd.read_excel(data_root +'EAVS_2018_for_Public_Release_Updates.xlsx', dtype=str)

#eXPLORE
print(eavs.shape)
eavs.describe()
eavs.head(30)

(6460, 407)


,FIPSCode,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13
0,0100100000,AUTAUGA COUNTY,ALABAMA,AL,39770,36391,3379,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,21117,8739,Data not available,Data not available,171,12207,Data not available,nan,nan,nan,nan,nan,nan,DO NOT HAVE DATA FOR ADDRESS CHANGES THAT CROS...,74,1452,2618,14485,646,45,0,172,35,USPS,253,POLLING PLACE,1246,PROBATE JUDGE'S OFFICE,91,29,721,1321,5900,299,31,0,76,21,USPS,6,POLLING PLACE,301,PROBATE JUDGE'S OFFICE,34,Data not available,Data not available,Data not available,Data not available,Data not available,Data not available,Data not available,Data not available,Data not available,nan,nan,nan,nan,nan,nan,2,8,1,91,48,2,0,8,1,USPS,0,POLLING PLACE,3,PROBATE JUDGE'S OFFICE,7,ALABAMA DOES NOT TRACK DUPLICATES THROUGH REGI...,4995,76,718,2800,1401,nan,nan,nan,nan,nan,nan,nan,2678,592,774,59,862,0,6,nan,nan,OTHER,385,nan,nan,nan,Data not available,Data not available,Data not available,VOTERS DO NOT REGISTER AS UOCAVA IN ALABAMA.,23,23,0,nan,0,0,0,nan,0,nan,9,4,5,1,1,0,8,3,5,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,1,1,0,1,1,0,0,0,0,0,0,0,nan,770,732,Data not available,Data not available,Data not available,38,nan,nan,nan,nan,nan,nan,nan,Does not apply,nan,728,nan,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,19814,nan,19089,Data not available,ALABAMA HAS IN PERSON ABSENTEE VOTING BUT WE D...,19,nan,19,19,Does not apply,1,0,1,nan,118,Data not available,ALABAMA DOES NOT TRACK THE NUMBER OF POLL 

In [11]:
#How many municipalities per each state? 
num_muni = eavs.groupby(['State_Abbr']).count()
num_muni.head()

,FIPSCode,Jurisdiction_Name,State_Full,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13
State_Abbr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AK,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
AL,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,67,6

In [12]:
# Clean Up
eavs = eavs.replace('Does not apply', None)
eavs = eavs.replace('Data not available', np.nan)
eavs = eavs.replace('NaN', None)
eavs = eavs.replace('Data not available', None)

In [13]:
# EAVS fields of Interest
eavs_vars = [ 'A1a', 'A1c', 'D1a','D2a','D2b', 'D4a', 'D4b', 'D6','D8f', 'D8g','D8a','F1b', 'F1a', 'F5c_1', 'F6c_1','F7c_1', 'F8c_1', 'F9c_1', 'F10c_1','F5c_2', 'F6c_2','F7c_2', 'F8c_2', 'F9c_2', 'F10c_2','F5c_3', 'F6c_3','F7c_3', 'F8c_3', 'F9c_3', 'F10c_3', 'D3a', 'C1a', 'C4a']

#Type Conversion
eavs[eavs_vars] = eavs[eavs_vars].astype('float64')

# All vars containing # of machines contain C_
machine_vars = [k for k in eavs_vars if 'c_' in k ]

###  Wisconsin Procesing

In [14]:
#Prepare to aggregate numeric columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

# Isolate Wisconsin town-level data  and remove from eavs for processing/aggregating
wi_eavs = eavs[eavs['State_Abbr']=='WI']
eavs =  eavs[eavs['State_Abbr']!='WI']

#Create Common Column; EAVS has long and short form FIPS Codes
eavs['short_fips'] =eavs.FIPSCode.apply(lambda x: str(x)[0:5])
wi_eavs['Jurisdiction Name'] = wi_eavs.Jurisdiction_Name.str.split("- ").str[1]
wi_eavs['State_Code'] = '55'
wi_eavs['Jurisdiction_Name'] = wi_eavs['Jurisdiction Name']


#Get True FIPS from external  source and merge
wi_names = pd.read_csv(data_root +'wi_counties.csv')
wi_names['Full_County_Name'] = wi_names['Full_County_Name'].astype(str)
wi = wi_eavs.merge(wi_names, how="left", left_on='Jurisdiction_Name', right_on='Full_County_Name')

#Clean and structure
wi = wi.dropna(subset=['short_fips'])
wi['short_fips'] = wi['short_fips'].astype(int)
wi['short_fips'] = wi['short_fips'].astype(str)
wi = wi.set_index(['short_fips', 'Jurisdiction_Name', 'State_Full', 'State_Abbr'])
wi = wi.select_dtypes(include=numerics)
wi.head()

A1a     A1c    C1a  C4a    D1a    D2a   D2b  D3a  D4a  D4b  D6  D8a  D8f  D8g    F1a    F1b  F5c_1  F5c_2  F5c_3  F6c_1  F6c_2  F6c_3  F7c_1  F7c_2  F7c_3  F8c_1  F8c_2  F8c_3  F9c_1  F9c_2  F9c_3  F10c_1  F10c_2  F10c_3
short_fips Jurisdiction_Name State_Full State_Abbr                                                                                                                                                                                                                                 
55083      OCONTO COUNTY     WISCONSIN  WI          1218.0  5636.0  118.0  0.0  919.0  795.0   0.0  1.0  1.0  1.0 NaN  NaN  NaN  NaN  919.0  795.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN
55067      LANGLADE COUNTY   WISCONSIN  WI           325.0  5636.0   15.0  0.0  266.0  247.0   4.0  1.0  1.0  1.0 NaN  NaN  NaN  NaN  266.0  247.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN
55001      ADAMS COUNTY      WISCONSIN  WI           905.0  5636.0   42.0  0.0  675.0  574.0  68.0  1.0  1.0  1.0 NaN  NaN  NaN  NaN  675.0  574.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN
                                        WI           740.0  5636.0   33.0  3.0  609.0  581.0   0.0  1.0  1.0  1.0 NaN  NaN  NaN  NaN  609.0  581.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN
55045      GREEN COUNTY      WISCONSIN  WI           337.0  5636.0   15.0  0.0  279.0  267.0   6.0  1.0  1.0  1.0 NaN  NaN  NaN  NaN  279.0  267.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN     NaN     NaN     NaN

In [15]:
#Prepare to Aggregate
sum_list = [ name for name in wi.columns if name != 'D9']
agg_dict = { var:'sum' for var in sum_list}
wi_agg = wi.reset_index().groupby(['short_fips', 'Jurisdiction_Name', 'State_Full', 'State_Abbr']).agg(agg_dict).reset_index()

#Check Output
wi_agg.head()

,short_fips,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1c,C1a,C4a,D1a,D2a,D2b,D3a,D4a,D4b,D6,D8a,D8f,D8g,F1a,F1b,F5c_1,F5c_2,F5c_3,F6c_1,F6c_2,F6c_3,F7c_1,F7c_2,F7c_3,F8c_1,F8c_2,F8c_3,F9c_1,F9c_2,F9c_3,F10c_1,F10c_2,F10c_3
0,55001,ADAMS COUNTY,WISCONSIN,WI,12029.0,107084.0,570.0,6.0,9529.0,8344.0,656.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,9529.0,8344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55003,ASHLAND COUNTY,WISCONSIN,WI,4744.0,84540.0,169.0,4.0,3547.0,3245.0,155.0,15.0,15.0,15.0,0.0,0.0,0.0,0.0,3547.0,3245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,55005,BARRON COUNTY,WISCONSIN,WI,25198.0,191624.0,954.0,21.0,18398.0,15838.0,1706.0,34.0,34.0,34.0,0.0,0.0,0.0,0.0,18398.0,15838.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,55007,BAYFIELD COUNTY,WISCONSIN,WI,11268.0,157808.0,638.0,12.0,8861.0,7757.0,518.0,28.0,28.0,28.0,0.0,0.0,0.0,0.0,8861.0,7757.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,55009,BROWN COUNTY,WISCONSIN,WI,134434.0,118356.0,7524.0,92.0,104158.0,78842.0,18568.0,87.0,62.0,62.0,0.0,0.0,0.0,0.0,104158.0,78842.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Add cleaned Wisconsin data to main dataframe
eavs = eavs.append(wi_agg, sort=False)

In [17]:
#Check
eavs[eavs['State_Abbr']=='WI'].short_fips[0:5]

0    55001
1    55003
2    55005
3    55007
4    55009
Name: short_fips, dtype: object

In [18]:
# Better Labels for EAVS fields
eavs['eligible_voters'] = eavs['A1a']
eavs['num_inactive'] = eavs['A1c']
eavs['num_same_day_pollworkers'] = eavs['D6']
eavs['num_machines'] = eavs[machine_vars].sum(axis=1, skipna=True)
eavs['in_person_voters'] = eavs['D2a'] #
eavs['num_pollworkers'] = np.where(pd.notna(eavs['num_same_day_pollworkers']), eavs['num_same_day_pollworkers'], eavs['D8a'])
eavs['num_same_day_locations'] = eavs['D4a']
eavs['num_pw_old'] = eavs['D8f'] + eavs['D8g']
eavs['voters_removed'] = eavs['A9a']
eavs['num_precincts'] = eavs['D3a']
eavs['total_mail_ballots'] =  eavs['C3a']
eavs['num_in_person'] = eavs['F1b'] 
eavs['num_in_person'] = eavs['F1b'] 


# Engineered Features
#Historical Characteristics
eavs['percent_in_person'] = eavs['F1b'] / eavs['F1a']
eavs['percent_inactive'] = eavs['num_inactive'] / eavs['eligible_voters']

eavs['percent_early_vote'] = eavs['D2b'] / eavs['D1a']
eavs['percent_rejected_ballot'] = eavs['C4a'] / eavs['C1a']

# Recode Difficulty Survey
difficulty = {
    'Very difficult':5,
    'Somewhat difficult':4,
    'Neither difficult nor easy':3,
    'Somewhat easy':2,
    'Very easy':1,
    'Not enough information to answer':None  
}
eavs['difficulty'] = eavs['D9'].map(difficulty)
#eavs.difficulty.unique()

#Resources
eavs['voter_per_location'] = eavs['eligible_voters'] / eavs['num_same_day_locations']
eavs['voter_per_machine'] =  eavs['eligible_voters']/ eavs['num_machines']
eavs['voter_per_pw'] = np.where(pd.notna(eavs['num_same_day_pollworkers']), eavs['eligible_voters'] / 
                    eavs['num_same_day_pollworkers'], eavs['eligible_voters']/ eavs['num_pollworkers'] )

#Blended
eavs['total_resources_per_voter'] = eavs['eligible_voters'] / (eavs['num_machines'] + eavs['num_pollworkers'])

#Demographic
eavs['num_pw_old'] = eavs['D8f'] + eavs['D8g']
eavs['frac_pw_old'] = eavs['num_pw_old'] / eavs['D8a']

In [19]:
eavs.total_resources_per_voter[0:5]

0    242.500000
1    242.007800
2    139.701613
3    158.266667
4    224.220339
Name: total_resources_per_voter, dtype: float64

### Merge all data across sources

In [20]:
# Cast FIPS to Integer
eavs.short_fips = eavs.short_fips.astype(str)
vote_by_fips.FIPSCode = vote_by_fips.FIPSCode.astype(str)

In [21]:
# Merge with Vote Time
df = eavs.merge(vote_by_fips, how="left", left_on='short_fips', right_on='FIPSCode')
df.head(50)

,FIPSCode_x,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13,short_fips,eligible_voters,num_inactive,num_same_day_pollworkers,num_machines,in_person_voters,num_pollworkers,num_same_day_locations,num_pw_old,voters_removed,num_precincts,total_mail_ballots,num_in_person,percent_in_person,percent_inactive,percent_early_vote,percent_rejected_ballot,difficulty,voter_per_location,voter_per_machine,voter_per_pw,total_resources_per_voter,frac_pw_old,FIPSCode_y,wait_time,wait_count
0,0100100000,AUTAUGA COUNTY,ALABAMA,AL,39770.0,36391,3379.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,21117,8739,NaN,NaN,171,12207,NaN,nan,nan,nan,nan,nan,nan,DO NOT HAVE DATA FOR ADDRESS CHANGES THAT CROS...,74,1452,2618,14485,646,45,0,172,35,USPS,253,POLLING PLACE,1246,PROBATE JUDGE'S OFFICE,91,29,721,1321,5900,299,31,0,76,21,USPS,6,POLLING PLACE,301,PROBATE JUDGE'S OFFICE,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,2,8,1,91,48,2,0,8,1,USPS,0,POLLING PLACE,3,PROBATE JUDGE'S OFFICE,7,ALABAMA DOES NOT TRACK DUPLICATES THROUGH REGI...,4995,76,718,2800,1401,nan,nan,nan,nan,nan,nan,nan,2678,592,774,59,862,0,6,nan,nan,OTHER,385,nan,nan,nan,NaN,NaN,NaN,VOTERS DO NOT REGISTER AS UOCAVA IN ALABAMA.,23,23,0,nan,0,0,0,nan,0,nan,9,4,5,1,1,0,8,3,5,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,1,1,0,1,1,0,0,0,0,0,0,0,nan,770.0,732,NaN,NaN,NaN,38,nan,nan,nan,nan,nan,nan,nan,Does not apply,nan,728,nan,4.0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,nan,nan,19814.0,nan,19089.0,NaN,ALABAMA 

In [22]:
df.isnull().sum().head()

FIPSCode_x           71
Jurisdiction_Name     0
State_Full            0
State_Abbr            0
A1a                  11
dtype: int64

## ACS Data

In [28]:
acs = pd.read_csv(data_root+ 'all_states_census.csv', dtype='str')

#Explore
print(acs.columns)
print(len(acs))
acs.head()

Index(['Unnamed: 0', 'index', 'DP05_0033E', 'DP05_0038E', 'DP05_0072E', 'DP03_0063E', 'DP02_0001E', 'DP03_0062E', 'DP05_0018E', 'DP05_0014E', 'DP05_0015E', 'DP05_0016E', 'DP05_0017E', 'fips_code', 'state_code', 'Name', 'FIPS State Numeric Code', 'Official USPS Code'], dtype='object')
3220


,Unnamed: 0,index,DP05_0033E,DP05_0038E,DP05_0072E,DP03_0063E,DP02_0001E,DP03_0062E,DP05_0018E,DP05_0014E,DP05_0015E,DP05_0016E,DP05_0017E,fips_code,state_code,Name,FIPS State Numeric Code,Official USPS Code
0,0,"Washington County, Mississippi: Summary level:...",47086,33982,579,49433,18299.0,30834,36.9,3291,4282,1961,710,28151,28,Mississippi,28,MS
1,1,"Perry County, Mississippi: Summary level: 050,...",12028,2368,150,55077,4563.0,39007,40.9,897,1247,589,236,28111,28,Mississippi,28,MS
2,2,"Choctaw County, Mississippi: Summary level: 05...",8321,2596,14,49162,3164.0,37203,44.0,644,979,547,214,28019,28,Mississippi,28,MS
3,3,"Itawamba County, Mississippi: Summary level: 0...",23480,1681,320,53717,8706.0,40510,40.0,1435,2250,1181,574,28057,28,Mississippi,28,MS
4,4,"Carroll County, Mississippi: Summary level: 05...",10129,3503,2,57919,3658.0,43060,47.0,623,1353,726,170,28015,28,Mississippi,28,MS


In [29]:
#Intelligble Variable Names
acs['total_population'] = acs['DP05_0033E'].astype(int)
acs['white_population'] = acs['DP05_0037E'].astype(int)
acs['black_population'] = acs['DP05_0038E'].astype(int)
acs['hispanic_population'] = acs['DP05_0072E'].astype(int)
acs['mean_household_income'] = acs['DP03_0063E'].astype(int)
acs['total_households'] = acs['DP02_0001E'].astype(float)
acs['median_household_income'] =  acs['DP03_0062E'].astype(int)
acs['median_age'] = acs['DP05_0018E'].astype(float)

#New Variables
acs['non_white_population'] = acs['total_population'] - acs['white_population']
acs['num_black_latino'] = acs['black_population'] + acs['hispanic_population']
acs['percent_minority'] = acs['non_white_population']  /  acs['total_population']
acs['num_senior'] =  (acs['DP05_0014E'].astype(float) + acs['DP05_0015E'].astype(float) + acs['DP05_0016E'].astype(float) + acs['DP05_0017E'].astype(float)) 
acs['percent_over_60'] = (acs['DP05_0014E'].astype(float) + acs['DP05_0015E'].astype(float) + acs['DP05_0016E'].astype(float) + acs['DP05_0017E'].astype(float)) / acs['total_population']

keep_census = ['fips_code','percent_minority', 'percent_over_60', 'median_household_income','total_households','median_age', 'num_senior', 'num_black_latino', 'total_population',]

In [33]:
full_df = df.merge(acs[keep_census], how="left", left_on="short_fips", right_on="fips_code")
full_df.head()

,FIPSCode_x,Jurisdiction_Name,State_Full,State_Abbr,A1a,A1b,A1c,A1Comments,A2a,A2Comments,A3a,A3b,A3c,A3d,A3e,A3f,A3g,A3h_Other,A3h,A3i_Other,A3i,A3j_Other,A3j,A3Comments,A4a,A4b,A4c,A4d,A4e,A4f,A4g,A4h,A4i,A4j_Other,A4j,A4k_Other,A4k,A4l_Other,A4l,A5a,A5b,A5c,A5d,A5e,A5f,A5g,A5h,A5i,A5j_Other,A5j,A5k_Other,A5k,A5l_Other,A5l,A6a,A6b,A6c,A6d,A6e,A6f,A6g,A6h,A6i,A6j_Other,A6j,A6k_Other,A6k,A6l_Other,A6l,A7a,A7b,A7c,A7d,A7e,A7f,A7g,A7h,A7i,A7j_Other,A7j,A7k_Other,A7k,A7l_Other,A7l,A4_A7Comments,A8a,A8b,A8c,A8d,A8e,A8f_Other,A8f,A8g_Other,A8g,A8h_Other,A8h,A8Comments,A9a,A9b,A9c,A9d,A9e,A9f,A9g,A9h_Other,A9h,A9i_Other,A9i,A9j_Other,A9j,A9Comments,B1a,B1b,B1c,B1Comments,B2a,B2b,B2c,B2Comments,B3a,B3b,B3c,B3Comments,B4a,B4Comments,B5a,B5b,B5c,B6a,B6b,B6c,B7a,B7b,B7c,B8a,B8b,B8c,B5_B8Comments,B9a,B9b,B9c,B10a,B10b,B10c,B11a,B11b,B11c,B12a,B12b,B12c,B9_B12Comments,B13a,B13b,B13c,B13d,B13Comments,B14a,B14b,B14c,B15a,B15b,B15c,B16a,B16b,B16c,B17a,B17b,B17c,B14_B17Comments,B18a,B18b,B18c,B19a,B19b,B19c,B20a,B20b,B20c,B21a,B21b,B21c,B22a,B22b,B22c,B22_Other,B18_B22Comments,B23a,B23b,B23c,B24a,B24b,B24c,B25a,B25b,B25c,B26a,B26b,B26c,B23_B26Comments,C1a,C1b,C1c,C1d,C1e,C1f,C1g_Other,C1g,C1h_Other,C1h,C1i_Other,C1i,C1Comments,C2a,C2Comments,C3a,C3Comments,C4a,C4b,C4c,C4d,C4e,C4f,C4g,C4h,C4i,C4j,C4k,C4l,C4m,C4n,C4o,C4p_Other,C4p,C4q_Other,C4q,C4r_Other,C4r,C4Comments,D1a,D1Comments,D2a,D2b,D2Comments,D3a,D3Comments,D4a,D4b,D4c,D5a,D5b,D5c,D4_D5Comments,D6,D7,D6_D7Comments,D8a,D8b,D8c,D8d,D8e,D8f,D8g,D8Comments,D9,D9Comments,E1a,E1b,E1c,E1d,E1e_Other,E1e,E1Comments,E2a,E2b,E2c,E2d,E2e,E2f,E2g,E2h,E2i,E2j,E2k_Other,E2k,E2l_Other,E2l,E2m_Other,E2m,E2Comments,F1a,F1b,F1c,F1d,F1e,F1f,F1g,F1h_Other,F1h,F1Comments,F2,F2_Other,F2Comments,F3a,F3b,F3c,F3d_Other,F3d,F4a,F4b,F4c,F4d_Other,F4d,F3_F4Comments,F5a,F5b_1,F5b_1other,F5c_1,F5b_2,F5b_2other,F5c_2,F5b_3,F5b_3other,F5c_3,F5d_1,F5d_2,F5d_3,F5d_4,F6a,F6b_1,F6b_1other,F6c_1,F6b_2,F6b_2other,F6c_2,F6b_3,F6b_3other,F6c_3,F6d_1,F6d_2,F6d_3,F6d_4,F7a,F7b_1,F7b_1other,F7c_1,F7b_2,F7b_2other,F7c_2,F7b_3,F7b_3other,F7c_3,F7d_1,F7d_2,F7d_3,F7d_4,F7d_5,F8a,F8b_1,F8b_1other,F8c_1,F8b_2,F8b_2other,F8c_2,F8b_3,F8b_3other,F8c_3,F8d_1,F8d_2,F8d_3,F8d_4,F8d_5,F9a,F9b_1other,F9c_1,F9b_2other,F9c_2,F9b_3other,F9c_3,F9d_1,F9d_2,F9d_3,F9d_4,F9d_5,F10a,F10b_1other,F10c_1,F10b_2other,F10c_2,F10b_3other,F10c_3,F10d_1,F10d_2,F10d_4,F11a,F11d_1,F11d_2,F11d_3,F11d_4,F11d_5,F5_F11Comments,F12a,F12b,F12c,F12d,F12e,F12Comments,F13,short_fips,eligible_voters,num_inactive,num_same_day_pollworkers,num_machines,in_person_voters,num_pollworkers,num_same_day_locations,num_pw_old,voters_removed,num_precincts,total_mail_ballots,num_in_person,percent_in_person,percent_inactive,percent_early_vote,percent_rejected_ballot,difficulty,voter_per_location,voter_per_machine,voter_per_pw,total_resources_per_voter,frac_pw_old,FIPSCode_y,wait_time,wait_count,fips_code,percent_minority,percent_over_60,median_household_income,total_households,median_age,num_senior,num_black_latino,total_population
0,0100100000,AUTAUGA COUNTY,ALABAMA,AL,39770.0,36391,3379.0,nan,Does not apply,ALABAMA DOES NOT HAVE SAME DAY REGISTRATION.,21117,8739,NaN,NaN,171,12207,NaN,nan,nan,nan,nan,nan,nan,DO NOT HAVE DATA FOR ADDRESS CHANGES THAT CROS...,74,1452,2618,14485,646,45,0,172,35,USPS,253,POLLING PLACE,1246,PROBATE JUDGE'S OFFICE,91,29,721,1321,5900,299,31,0,76,21,USPS,6,POLLING PLACE,301,PROBATE JUDGE'S OFFICE,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,2,8,1,91,48,2,0,8,1,USPS,0,POLLING PLACE,3,PROBATE JUDGE'S OFFICE,7,ALABAMA DOES NOT TRACK DUPLICATES THROUGH REGI...,4995,76,718,2800,1401,nan,nan,nan,nan,nan,nan,nan,2678,592,774,59,862,0,6,nan,nan,OTHER,385,nan,nan,nan,NaN,NaN,NaN,VOTERS DO NOT REGISTER AS UOCAVA IN ALABAMA.,23,23,0,nan,0,0,0,nan,0,nan,9,4,5,1,1,0,8,3,5,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,nan,2,2,0,2,2,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,1,1,0,1,1,0,0,0,0,0,0,0,nan,770.0,732,NaN,NaN,NaN,38,nan,nan

### Understand EAVS Municipalities

While most municipalities submitted to EAVS are at the county level, there are some exceptions. Some states, like Wisconsin, submit these forms at the township or village level.

In [34]:
#Understand How States Actually Submitted Their EAVS data
eavs_muni = full_df[['fips_code', 'State_Abbr']].groupby(['State_Abbr']).count().reset_index()
census_counties = acs[['fips_code','Official USPS Code']].groupby(['Official USPS Code']).count().reset_index()

#Create table of differences
get_disconnect = census_counties.merge(eavs_muni, how="left", left_on='Official USPS Code', right_on='State_Abbr')
get_disconnect = get_disconnect.drop(['State_Abbr'], axis=1)
get_disconnect.columns = ['State', 'Correct_Census_Count', 'EAVS_Count']
get_disconnect['county_match'] = get_disconnect['Correct_Census_Count'] == get_disconnect['EAVS_Count']

get_disconnect.head()

,State,Correct_Census_Count,EAVS_Count,county_match
0,AK,29,0,False
1,AL,67,67,True
2,AR,75,75,True
3,AZ,15,15,True
4,CA,58,58,True


In [54]:
true_matches = get_disconnect.county_match.sum()
print('There are', 50-true_matches, 'states without county-level info here.')

trouble_states = get_disconnect[get_disconnect['county_match']==False]['State']
trouble_states = trouble_states.drop(labels=[48]) #WI is repaired

There are 9 states without county-level info here.


### Export Only Features of Interest


In [69]:
# Units and Features of Interest
key_states = ['TX','OH', 'AZ', 'PA', 'WI', 'NC','FL', 'MI']

# #Features we want to keep in the final merge / Drop Excess Columns 
keep_vars = [
    #Main Indicators
    'State_Full','short_fips', 'State_Abbr', 'Jurisdiction_Name', 'non_white_population','num_senior', 'num_pw_old','difficulty', 'num_in_person','num_inactive',
   'voter_per_location', 'voter_per_machine', 'voter_per_pw','wait_time','wait_count', 
     #Other Data
             'num_same_day_locations', 'num_machines', 'num_pollworkers', 'num_pw_old', 'voters_removed', 'num_precincts', 'total_mail_ballots', 'percent_early_vote', 'percent_rejected_ballot',  'median_age', 'median_household_income', 'total_population',
             'eligible_voters', 'frac_pw_old', 'percent_in_person', 'percent_inactive', 'percent_minority', 'percent_over_60']


# Features for Looping and Riskiness
risk_feats = ['State_Full','short_fips', 'State_Abbr', 'Jurisdiction_Name', 'num_black_latino','num_senior', 'num_pw_old','difficulty', 'num_in_person','num_inactive',
   'voter_per_location', 'voter_per_machine', 'voter_per_pw','wait_time','wait_count']

# DF for Key States and Risk Variables
clean_df = full_df[~full_df['State_Abbr'].isin(trouble_states)][keep_vars]
key_df = full_df[full_df['State_Abbr'].isin(key_states)][keep_vars]
key_df.head()


,State_Full,short_fips,State_Abbr,Jurisdiction_Name,num_black_latino,num_senior,num_pw_old,difficulty,num_in_person,num_inactive,voter_per_location,voter_per_machine,voter_per_pw,wait_time,wait_count,num_same_day_locations,num_machines,num_pollworkers,num_pw_old,voters_removed,num_precincts,total_mail_ballots,percent_early_vote,percent_rejected_ballot,median_age,median_household_income,total_population,eligible_voters,frac_pw_old,percent_in_person,percent_inactive,percent_minority,percent_over_60
69,ARIZONA,04001,AZ,APACHE COUNTY,3859.0,14758.0,99.0,4.0,13440.0,4971.0,1306.666667,1193.043478,197.410072,NaN,NaN,42.0,46.0,278.0,99.0,4930,44.0,-88,0.034939,0.017120,34.7,32963.0,71522.0,54880.0,0.356115,0.525842,0.090579,0.053955,0.206342
70,ARIZONA,04003,AZ,COCHISE COUNTY,44041.0,36129.0,77.0,4.0,8591.0,16381.0,5131.470588,545.218750,510.146199,NaN,NaN,17.0,160.0,171.0,77.0,8539,49.0,34144,0.015873,0.016885,40.6,48649.0,126279.0,87235.0,0.450292,0.187058,0.187780,0.348759,0.286105
71,ARIZONA,04005,AZ,COCONINO COUNTY,17803.0,24202.0,161.0,4.0,15391.0,15500.0,1795.254545,836.771186,281.307692,NaN,NaN,55.0,118.0,351.0,161.0,15114,71.0,37112,0.032780,0.005270,30.6,57616.0,140217.0,98739.0,0.458689,0.275095,0.156980,0.126967,0.172604
72,ARIZONA,04007,AZ,GILA COUNTY,9515.0,19272.0,161.0,5.0,3635.0,7433.0,2178.470588,1157.312500,356.096154,NaN,NaN,17.0,32.0,104.0,161.0,4539,39.0,17144,0.037311,0.025941,49.7,42092.0,53400.0,37034.0,1.548077,0.174550,0.200707,0.178184,0.360899
73,ARIZONA,04009,AZ,GRAHAM COUNTY,12587.0,7083.0,25.0,3.0,3469.0,3250.0,2350.666667,1511.142857,480.818182,NaN,NaN,9.0,14.0,44.0,25.0,2903,22.0,7204,0.097627,0.015295,33.3,51352.0,37879.0,21156.0,0.568182,0.321621,0.153621,0.332295,0.186990


In [70]:
# Output Final Risk Indicators Datarame
import pickle

clean_df.to_csv(data_root + 'all_states_risk_data.csv')
key_df.to_csv(data_root+'key_states_risk_data.csv')
key_df.to_pickle('key_states.pkl')
clean_df.to_pickle('all_df.pkl')

### Visualize Missingness

In [71]:
# Understand Which States Are Missing Which Values
track_state_nas = key_df[risk_feats].set_index('State_Abbr').isna().sum(level=0)
num_counties = key_df[risk_feats].groupby(['State_Abbr']).count()
percent_missing = track_state_nas / num_counties
#sns.heatmap(percent_missing, annot=True)
percent_missing.to_csv('missing_matrix.csv')
track_state_nas.to_csv('count_nas_by_state.csv')
#track_state_nas.to_csv

In [ ]:

track_all_nas = full_df[risk_feats].set_index('State_Abbr').isna().sum(level=0)
num_counties = full_df[risk_feats].groupby(['State_Abbr']).count()
percent_missing = track_all_nas / num_counties
percent_missing.to_csv('toolkit_ready_states.csv')